# Laboratory exercise 6

The notebook contains exercises connected to auditory exercise 7. For any questions feel free to contact assistant: eda.jovicic@fer.hr

The main task of this notebook is to make predictions using supported learning with the Scikit Learn library. The goal is to predict the grade in Math considering other features of the student.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

1. Load the cleaned dataset from the first exercise. If you haven't saved the dataset, rerun the exercise and save the final dataset.

In [42]:
#Učitavanje dataseta
import pandas as pd
students = pd.read_csv("studentslab.csv")
students

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,2,1,standard,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0
1,female,2,5,standard,1,1,2,1,0.0,2,0,69,90,88,1.0,0.0,0.0,1.0
2,female,1,3,standard,1,2,2,1,4.0,1,1,87,93,91,1.0,0.0,0.0,1.0
3,male,0,0,free/reduced,1,1,0,0,1.0,2,0,45,56,42,0.0,1.0,1.0,0.0
4,male,2,5,standard,1,1,2,1,0.0,1,0,76,78,75,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,female,3,2,standard,1,2,2,0,2.0,1,0,59,61,65,1.0,0.0,0.0,1.0
29113,male,4,2,standard,1,2,1,0,1.0,0,0,58,53,51,0.0,1.0,0.0,1.0
29114,female,2,2,free/reduced,0,1,2,0,1.0,0,0,61,70,67,1.0,0.0,1.0,0.0
29115,female,3,0,standard,0,1,1,0,3.0,1,0,82,90,93,1.0,0.0,0.0,1.0


2. Transform the MathScore feature into grades (1-5) using the following scoring system:

|   MathScore |  Grade  |
| ----------- | ------- |
|  88 - 100   |    5    |
|  75 - 87    |    4    |
|  63 - 74    |    3    |
|  50 - 62    |    2    |
|   0 - 49    |    1    |

In [43]:
import pandas as pd

#Definiranje
bins = [-1, 49, 62, 74, 87, 100]
grades = [1, 2, 3, 4, 5]

#Stupac 'Grade'
students['Grade'] = pd.cut(students['MathScore'], bins=bins, labels=grades, right=False)

print(students[['MathScore', 'Grade']])


       MathScore Grade
0             71     3
1             69     3
2             87     5
3             45     1
4             76     4
...          ...   ...
29112         59     2
29113         58     2
29114         61     2
29115         82     4
29116         64     3

[29117 rows x 2 columns]


3. Divide the dataset into features (X) and predictions (y). For features we will use all the columns except MathScore, ReadingScore and WritingScore. For predictions we will use the MathScore column. Split the dataset into training and testing sets. The split should be done in a  70-30% ratio.

In [44]:
#X (sve osim MathScore, ReadingScore, WritingScore) and y (MathScore)
import pandas as pd
from sklearn.model_selection import train_test_split

col = ['EthnicGroup', 'ParentEduc', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'NrSiblings', 'TransportMeans', 'WklyStudyHours']
X = students[col]
y = students.MathScore

X.describe()

,EthnicGroup,ParentEduc,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours
count,29117.000000,29117.000000,29117.000000,29117.000000,29117.000000,29117.000000,29117.000000,29117.000000,29117.000000
mean,2.172957,3.080881,0.675276,1.109077,1.372635,0.642030,2.147714,0.730501,0.607995
std,1.105460,2.190324,0.468280,0.676667,0.709069,0.479411,1.428635,0.633428,0.760503
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
50%,2.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000
75%,3.000000,5.000000,1.000000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000
max,4.000000,6.000000,1.000000,3.000000,2.000000,1.000000,26.000000,2.000000,2.000000


In [45]:
#Razdvajanje dataseta na train i test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (20381, 9)
X_test shape: (8736, 9)
y_train shape: (20381,)
y_test shape: (8736,)


4. Create a Random Forest model, with max_depth=5 and n_estimators=20. Train the model using training set, and then test it on testing set. Display the confusion matrix. Show precision, recall and F1 score for all grades. 

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

#Isključivanje nenumeričke stupce i ciljnu varijablu 'MathScore'
#X = students.drop(columns=['MathScore', 'ReadingScore', 'WritingScore', 'TestPrep', 'ParentMaritalStatus'])
#y = students['MathScore']

#categorical_cols = ['TestPrep', 'ParentMaritalStatus']  # Update with your categorical columns
#X_categorical = pd.get_dummies(students[categorical_cols])

#X = pd.concat([X, X_categorical], axis=1)

#rows_with_nan = y[y.isnull()].index

#X = X.drop(rows_with_nan)
#y = y.drop(rows_with_nan)

clf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=20)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) #Ni ovo ne ispada dobro

#Razdvajanje dataseta na train i test (70% train, 30% test)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Stvaranje i treniranje modela Random Forest
#rf_model = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=42)
#rf_model.fit(X_train, y_train)

#Predikcije
#y_pred = rf_model.predict(X_test)

#conf_matrix = confusion_matrix(y_test, y_pred)
#print("Confusion Matrix:")
#print(conf_matrix)

#Izračunavanje preciznosti, opoziva i F1 rezultata za sve ocjene pomoću classification_report
#grades = sorted(pd.unique(y_test))
#report = classification_report(y_test, y_pred, labels=grades)
#print("\nClassification Report:")
#print(report)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  2]
 [ 0  0  0 ...  0  0 14]]
              precision    recall  f1-score   support

           7       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         3
          21       0.00      0.00      0.00         2
          22       0.00      0.00      0.00         2
          23       0.00      0.00      0.00         3
          24       0.00      0.00      0.00         6
          25       0.00      0.00      0.00         7
          26       0.00      0.00      0.00        10
          27       0.00      0.00

C:\Users\imlau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\imlau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\imlau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5. The accuracy of our model is not the best. The reason could be having too many possible classes (grades). Let's transform the data again, but this time, instead of predicting grades, we want to predict whether the student will pass (grades 2, 3, 4 and 5) or fail (grade 1) math. After transforming the MathScore accordingly (0 - failed, 1 - passed), repeat task 4 and compare the results.

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd

#Pretvaranje stupca 'MathScore' u prolaz (1) ili pad (0)
#students['MathOutcome'] = students['MathScore'].apply(lambda x: 1 if x > 1 else 0)

#Isključivanje nenumeričkih stupaca i ciljne varijable 'MathScore', 'ReadingScore', 'WritingScore'
#X = students.drop(columns=['MathScore', 'ReadingScore', 'WritingScore', 'MathOutcome', 'TestPrep', 'ParentMaritalStatus'])
#y = students['MathOutcome']

#categorical_cols = ['TestPrep', 'ParentMaritalStatus']  # Update with your categorical columns
#X_categorical = pd.get_dummies(students[categorical_cols])

#X = pd.concat([X, X_categorical], axis=1)

#Razdvajanje dataseta na train i test (70% train, 30% test)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

students["MathScore"]=pd.cut(x=students["MathScore"], bins=[0, 1, 5], labels=[0,1])
students["MathScore"] #Ni ovo ne ispada dobro

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
29112    NaN
29113    NaN
29114    NaN
29115    NaN
29116    NaN
Name: MathScore, Length: 29117, dtype: category
Categories (2, int64): [0 < 1]

In [39]:
#rf_model = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=42)
#rf_model.fit(X_train, y_train)

#Predikcije
#y_pred = rf_model.predict(X_test)

#conf_matrix = confusion_matrix(y_test, y_pred)
#print("Confusion Matrix:")
#print(conf_matrix)

#report = classification_report(y_test, y_pred)
#print("\nClassification Report:")
#print(report)

y = students.MathScore
#splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
#Random Forest model
clf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=20)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) #Ni ovo ne ispada dobro

ValueError: Input y contains NaN.

6. Compare the results. Did our model work better in the first case or the second? Explain why and suggest a way to improve it.